<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#План-работы-на-заключительный-этап" data-toc-modified-id="План-работы-на-заключительный-этап-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>План работы на заключительный этап</a></span></li><li><span><a href="#Загрузка-данных-и-библиотек" data-toc-modified-id="Загрузка-данных-и-библиотек-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка данных и библиотек</a></span></li><li><span><a href="#Optimization" data-toc-modified-id="Optimization-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Optimization</a></span><ul class="toc-item"><li><span><a href="#Data-preparation" data-toc-modified-id="Data-preparation-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Data preparation</a></span></li><li><span><a href="#Maximization" data-toc-modified-id="Maximization-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Maximization</a></span></li><li><span><a href="#Schedule" data-toc-modified-id="Schedule-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Schedule</a></span></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#Eval-own-algo" data-toc-modified-id="Eval-own-algo-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Eval own algo</a></span></li><li><span><a href="#Baseline-amount-prediction" data-toc-modified-id="Baseline-amount-prediction-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Baseline amount prediction</a></span><ul class="toc-item"><li><span><a href="#Data-preparation" data-toc-modified-id="Data-preparation-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Data preparation</a></span></li></ul></li><li><span><a href="#Eval-baseline" data-toc-modified-id="Eval-baseline-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Eval baseline</a></span></li></ul></li><li><span><a href="#Final-result" data-toc-modified-id="Final-result-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Final result</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# План работы на заключительный этап

1) Создать оптимизационную функцию по максимизации прибыли за 3 дня, учитывающую все ограничения

2) Найти лучшую цену за каждые 3 дня, предсказав спрос с помощью предобученной модели и определить прибыль

3) Подставить параметры в бейзлайн цен, определить спрос с этой ценой там и посчитать прибыль

4) Сравнить прибыль полученную данным образом с бейзлайном

# Загрузка данных и библиотек

In [1]:
import pandas as pd
import numpy as np

from catboost import CatBoostRegressor

from tqdm.notebook import tqdm

RAND = 10

In [2]:
df_modelling = pd.read_csv('data/price_forecasting/merged_forecast_df.csv',
                           parse_dates=['date'])
df_baseline = pd.read_parquet('data/raw/df_baseline.parquet')

In [3]:
catb_model = CatBoostRegressor()
catb_model.load_model('C:/Users/Tortik/Desktop/gpn/model/model_catb.cb')

In [4]:
catb_model.feature_names_

['place',
 'product',
 'competitors_mean_price',
 'cost',
 'hot',
 'rain',
 'snow',
 'month_name',
 'season',
 'own_price',
 'price_comparison']

# Optimization
## Data preparation

In [5]:
# Добавим признаки сезонности и месяца для модели
df_modelling['month_name'] = df_modelling['date'].dt.month_name()
df_modelling['season'] = pd.to_datetime(df_modelling['date']).dt.month.map({
    1:
    'Winter',
    2:
    'Winter',
    3:
    'Spring',
    4:
    'Spring',
    5:
    'Spring',
    6:
    'Summer',
    7:
    'Summer',
    8:
    'Summer',
    9:
    'Fall',
    10:
    'Fall',
    11:
    'Fall',
    12:
    'Winter'
})

In [6]:
df_modelling

,place,product,date,cost,competitors_mean_price,hot,rain,snow,month_name,season
0,Анор Лондо,Целебные травы,2218-09-28,6.950551,9.764069,0,0,0,September,Fall
1,Анор Лондо,Целебные травы,2218-09-29,6.945553,9.796939,1,0,0,September,Fall
2,Анор Лондо,Целебные травы,2218-09-30,6.926053,9.648104,0,0,0,September,Fall
3,Анор Лондо,Целебные травы,2218-10-01,6.902003,9.573142,0,1,0,October,Fall
4,Анор Лондо,Целебные травы,2218-10-02,6.955854,9.638593,0,0,0,October,Fall
...,...,...,...,...,...,...,...,...,...,...
1345,Фалькония,Эстус,2218-12-22,5.309055,15.655683,0,0,0,December,Winter
1346,Фалькония,Эстус,2218-12-23,5.264601,15.582679,0,0,0,December,Winter
1347,Фалькония,Эстус,2218-12-24,5.285202,15.597812,0,0,0,December,Winter
1348,Фалькония,Эстус,2218-12-25,5.280859,15.655546,0,0,1,December,Winter


- Т.к. высшим силам не нужно показывать прогноз на цены конкурента и в данном случае посчитано среднее, не будем накладывать ограничения на их цену

In [7]:
def calculate_total_profit(prices: list, amounts: list, costs: list) -> float:
    """
    Рассчитывает итоговую прибыль на основе списков цен, количеств и затрат.

    Параметры:
    - prices (list): Список цен продажи продукта.
    - amounts (list): Список количеств проданных продуктов.
    - costs (list): Список затрат на производство каждой единицы продукта.

    Возвращает:
    float: Итоговая прибыль, которая представляет собой сумму выручки минус сумма затрат.
    """
    total_profit = sum((price * amount) - (cost * amount)
                       for price, amount, cost in zip(prices, amounts, costs))
    return total_profit

In [8]:
cols = df_modelling.select_dtypes('object').columns
df_modelling[cols] = df_modelling[cols].astype("category")

In [9]:
df_modelling.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   place                   1350 non-null   category      
 1   product                 1350 non-null   category      
 2   date                    1350 non-null   datetime64[ns]
 3   cost                    1350 non-null   float64       
 4   competitors_mean_price  1350 non-null   float64       
 5   hot                     1350 non-null   int64         
 6   rain                    1350 non-null   int64         
 7   snow                    1350 non-null   int64         
 8   month_name              1350 non-null   category      
 9   season                  1350 non-null   category      
dtypes: category(4), datetime64[ns](1), float64(2), int64(3)
memory usage: 69.3 KB


## Maximization

In [10]:
def find_optimal_price(data: pd.DataFrame) -> pd.DataFrame:
    """
    Максимизируя прибыль оптимизирует цены на продукты в DataFrame для каждого города по продукту, в интервале по 3 дня.
    Определяет прибыль, спрос и чья цена выше.

    Parameters:
    - data (pd.DataFrame): Исходные данные, содержащие информацию о продуктах и ценах.

    Returns:
    - pd.DataFrame: DataFrame с оптимизированными ценами и объемами продаж.
    """
    # Итерируемся по дням с шагом 3
    for i in tqdm(range(0, len(data), 3)):
        i = round(i)
        # Рассчитываем верхнюю границу конкурентных цен
        max_comp = round(data['competitors_mean_price'].iloc[i:i + 3],
                         2).mean() * 1.2
        if i == 0:
            # Если первый интервал, устанавливаем минимальную цену в виде максимальных издержек
            min_price = round(data['cost'].iloc[:3], 2).max()
            max_price = max_comp
            data['own_price'] = None
        else:
            # Если не первый интервал, устанавливаем минимальную цену из предыдущего интервала
            min_price = pd.to_numeric(data.loc[i - 3, 'own_price'])
            # Смотрим максимальную цену по конкуренту и максимальную возможность её повысить
            max_price = min((min_price + 1), max_comp)
            # Корректируем границы, если минимальная цена превышает максимальную
            if min_price > max_price:
                max_comp = round(data['competitors_mean_price'].iloc[i:i + 3],
                                 2).max() * 1.2
                max_price = min((min_price + 1), max_comp)

        # {price: profit for 3 days}
        hashmap = {}
        # На всякий случай для перестраховки
        if min_price - 1 > max_price + 0.01:
            print(i)
            print(max_price)
            print(min_price)
            print('error')
        # Перебираем цены в заданном диапазоне
        for price in np.arange(min_price - 1, max_price + 0.01, 0.01):
            # Приводим столбец 'own_price' к числовому типу
            data['own_price'] = pd.to_numeric(data['own_price'],
                                              errors='coerce')
            # Устанавливаем цену для текущего интервала
            data.loc[i:i + 2, 'own_price'] = float(price)

            # Сравниваем цены с конкурентами, добавляя признак для модели
            data.loc[i:i + 2, 'price_comparison'] = np.where(
                data['own_price'].iloc[i:i + 3] >
                data['competitors_mean_price'].iloc[i:i + 3], 'Higher',
                'Lower')
            data['price_comparison'] = pd.Categorical(
                data['price_comparison'],
                categories=['Higher', 'Lower'],
                ordered=True)

            # Предсказываем объемы продаж
            data.loc[i:i + 2, 'amount'] = catb_model.predict(
                data.drop(['date'], axis=1).iloc[i:i + 3])
            # Рассчитываем прибыль для данной цены за 3 дня
            profit = calculate_total_profit(data['own_price'].iloc[i:i + 3],
                                            data['amount'].iloc[i:i + 3],
                                            data['cost'].iloc[i:i + 3])
            # Запоминаем цену и её прибыль за 3 дня
            hashmap[f'{price}'] = profit

        # Выбираем лучшую цену на основе максимальной прибыли
        best_price = max(hashmap, key=hashmap.get)
        data.loc[i:i + 2, 'own_price'] = best_price
        # Пересчитываем объемы продаж для лучшей цены
        data.loc[i:i + 2, 'amount'] = catb_model.predict(
            data.drop(['date'], axis=1).iloc[i:i + 3])
    return data

In [11]:
df_modelling

,place,product,date,cost,competitors_mean_price,hot,rain,snow,month_name,season
0,Анор Лондо,Целебные травы,2218-09-28,6.950551,9.764069,0,0,0,September,Fall
1,Анор Лондо,Целебные травы,2218-09-29,6.945553,9.796939,1,0,0,September,Fall
2,Анор Лондо,Целебные травы,2218-09-30,6.926053,9.648104,0,0,0,September,Fall
3,Анор Лондо,Целебные травы,2218-10-01,6.902003,9.573142,0,1,0,October,Fall
4,Анор Лондо,Целебные травы,2218-10-02,6.955854,9.638593,0,0,0,October,Fall
...,...,...,...,...,...,...,...,...,...,...
1345,Фалькония,Эстус,2218-12-22,5.309055,15.655683,0,0,0,December,Winter
1346,Фалькония,Эстус,2218-12-23,5.264601,15.582679,0,0,0,December,Winter
1347,Фалькония,Эстус,2218-12-24,5.285202,15.597812,0,0,0,December,Winter
1348,Фалькония,Эстус,2218-12-25,5.280859,15.655546,0,0,1,December,Winter


In [12]:
# Группируем данные по месту и продукту
grouped = df_modelling.groupby(['place', 'product'])

# Создаем новый DataFrame для хранения результатов
result_df = pd.DataFrame()

for (place, product), group in grouped:
    print({place}, {product})
    group.reset_index(drop=True, inplace=True)
    group = find_optimal_price(group)

    result_df = pd.concat([result_df, group], ignore_index=True)
result_df

{'Анор Лондо'} {'Целебные травы'}


  0%|          | 0/30 [00:00<?, ?it/s]

{'Анор Лондо'} {'Эльфийская пыльца'}


  0%|          | 0/30 [00:00<?, ?it/s]

{'Анор Лондо'} {'Эстус'}


...

{'Фалькония'} {'Эльфийская пыльца'}


  0%|          | 0/30 [00:00<?, ?it/s]

{'Фалькония'} {'Эстус'}


  0%|          | 0/30 [00:00<?, ?it/s]

,place,product,date,cost,competitors_mean_price,hot,rain,snow,month_name,season,own_price,price_comparison,amount
0,Анор Лондо,Целебные травы,2218-09-28,6.950551,9.764069,0,0,0,September,Fall,11.69,Higher,36.513806
1,Анор Лондо,Целебные травы,2218-09-29,6.945553,9.796939,1,0,0,September,Fall,11.69,Higher,48.987010
2,Анор Лондо,Целебные травы,2218-09-30,6.926053,9.648104,0,0,0,September,Fall,11.69,Higher,36.162428
3,Анор Лондо,Целебные травы,2218-10-01,6.902003,9.573142,0,1,0,October,Fall,11.55,Higher,54.477336
4,Анор Лондо,Целебные травы,2218-10-02,6.955854,9.638593,0,0,0,October,Fall,11.55,Higher,41.017364
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,Фалькония,Эстус,2218-12-22,5.309055,15.655683,0,0,0,December,Winter,18.67,Higher,51.163281
1346,Фалькония,Эстус,2218-12-23,5.264601,15.582679,0,0,0,December,Winter,18.67,Higher,50.906692
1347,Фалькония,Эстус,2218-12-24,5.285202,15.597812,0,0,0,December,Winter,18.67000000000436,Higher,50.878696
1348,Фалькония,Эстус,2218-12-25,5.280859,15.655546,0,0,1,December,Winter,18.67000000000436,Higher,61.582752


In [13]:
result_df['own_price'] = pd.to_numeric(result_df['own_price'], errors='coerce')

In [14]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   place                   1350 non-null   category      
 1   product                 1350 non-null   category      
 2   date                    1350 non-null   datetime64[ns]
 3   cost                    1350 non-null   float64       
 4   competitors_mean_price  1350 non-null   float64       
 5   hot                     1350 non-null   int64         
 6   rain                    1350 non-null   int64         
 7   snow                    1350 non-null   int64         
 8   month_name              1350 non-null   category      
 9   season                  1350 non-null   category      
 10  own_price               1350 non-null   float64       
 11  price_comparison        1350 non-null   category      
 12  amount                  1350 non-null   float64 

In [15]:
result_df['profit'] = result_df['own_price'] * result_df['amount'] - result_df[
    'cost'] * result_df['amount']

In [16]:
result_df.drop(['date'], axis=1).describe()

,cost,competitors_mean_price,hot,rain,snow,own_price,amount,profit
count,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000
mean,9.686665,17.632133,0.171111,0.166667,0.175556,21.114444,51.297499,585.032381
std,4.072719,6.481336,0.376746,0.372816,0.380583,7.794988,7.989764,265.023201
min,4.030448,8.438943,0.000000,0.000000,0.000000,10.000000,28.648133,147.713834
25%,6.175143,11.707288,0.000000,0.000000,0.000000,14.080000,45.877679,381.066416
50%,8.763666,16.631793,0.000000,0.000000,0.000000,19.940000,50.679574,509.522340
75%,12.351681,20.479277,0.000000,0.000000,0.000000,24.590000,57.611257,771.858903
max,21.788956,33.161279,1.000000,1.000000,1.000000,39.780000,71.869452,1501.388646


- *Функция успешно отработала, теперь создаём расписание цен на 90 дней*

## Schedule

In [17]:
# Инициализация пустого списка для данных
price_data = []

# Группировка DataFrame по городу и продукту
grouped = result_df.groupby(['place', 'product'])

# Итерация по группам
for group_name, group_data in grouped:
    # Сброс индекса внутри каждой группы
    group_data = group_data.reset_index(drop=True)

    # Итерация по строкам внутри каждой группы
    for i in range(len(group_data)):
        # Получение информации о месте, продукте и цене
        place = group_data.loc[i, 'place']
        product = group_data.loc[i, 'product']
        price = group_data.loc[i, 'own_price']

        # Создание словаря для хранения данных
        data = {
            'day_num': i + 1,  # Так как индексация начинается с 0, добавляем 1
            'place': place,
            'product': product,
            'price': price
        }

        # Добавление словаря в список
        price_data.append(data)

# Создание DataFrame из списка
price_schedule = pd.DataFrame(price_data)

# Вывод DataFrame
price_schedule

,day_num,place,product,price
0,1,Анор Лондо,Целебные травы,11.69
1,2,Анор Лондо,Целебные травы,11.69
2,3,Анор Лондо,Целебные травы,11.69
3,4,Анор Лондо,Целебные травы,11.55
4,5,Анор Лондо,Целебные травы,11.55
...,...,...,...,...
1345,86,Фалькония,Эстус,18.67
1346,87,Фалькония,Эстус,18.67
1347,88,Фалькония,Эстус,18.67
1348,89,Фалькония,Эстус,18.67


In [18]:
# сверим с бейзлайном
df_baseline

,day_num,place,product,price
0,1,Анор Лондо,Целебные травы,13.63
1,2,Анор Лондо,Целебные травы,13.63
2,3,Анор Лондо,Целебные травы,13.63
3,4,Анор Лондо,Целебные травы,13.63
4,5,Анор Лондо,Целебные травы,13.63
...,...,...,...,...
85,86,Фалькония,Эстус,9.28
86,87,Фалькония,Эстус,9.28
87,88,Фалькония,Эстус,9.28
88,89,Фалькония,Эстус,9.28


In [19]:
# сохраняем расписание цен
# на всякий случай в два формата для удобства просмотра
price_schedule.to_csv(
    r'C:\Users\Tortik\Desktop\gpn\data\price_schedule\price_schedule.csv')
price_schedule.to_parquet(
    r'C:\Users\Tortik\Desktop\gpn\data\price_schedule\price_schedule.parquet')

# Evaluation

## Eval own algo

In [20]:
evaluation_df = pd.DataFrame({
    "profit_own_algo": [result_df['profit'].sum()],
    "profit_baseline": [None]
})

In [21]:
result_df['profit'].sum()

789793.7149418455

In [22]:
evaluation_df

,profit_own_algo,profit_baseline
0,789793.714942,None


- *Проверим спрос для бейзлайна*

## Baseline amount prediction

### Data preparation

- *Добавим необходимые и предсказанные на 90 дней признаки для определения спроса и прибыли*

In [23]:
grouped = df_baseline.groupby(['place', 'product'])

# Создаем новый DataFrame для хранения результатов
baseline_date_df = pd.DataFrame()

for (place, product), group in grouped:
    date_range = pd.date_range(start='2218-09-28', end='2218-12-26')
    group['date'] = date_range
    baseline_date_df = pd.concat([baseline_date_df, group], ignore_index=True)

baseline_date_df

,day_num,place,product,price,date
0,1,Анор Лондо,Целебные травы,13.63,2218-09-28
1,2,Анор Лондо,Целебные травы,13.63,2218-09-29
2,3,Анор Лондо,Целебные травы,13.63,2218-09-30
3,4,Анор Лондо,Целебные травы,13.63,2218-10-01
4,5,Анор Лондо,Целебные травы,13.63,2218-10-02
...,...,...,...,...,...
1345,86,Фалькония,Эстус,9.28,2218-12-22
1346,87,Фалькония,Эстус,9.28,2218-12-23
1347,88,Фалькония,Эстус,9.28,2218-12-24
1348,89,Фалькония,Эстус,9.28,2218-12-25


In [24]:
baseline_date_df = baseline_date_df.drop(['day_num'], axis=1)

In [25]:
baseline_date_df.rename(columns={'price': 'own_price'}, inplace=True)

In [26]:
baseline_date_df

,place,product,own_price,date
0,Анор Лондо,Целебные травы,13.63,2218-09-28
1,Анор Лондо,Целебные травы,13.63,2218-09-29
2,Анор Лондо,Целебные травы,13.63,2218-09-30
3,Анор Лондо,Целебные травы,13.63,2218-10-01
4,Анор Лондо,Целебные травы,13.63,2218-10-02
...,...,...,...,...
1345,Фалькония,Эстус,9.28,2218-12-22
1346,Фалькония,Эстус,9.28,2218-12-23
1347,Фалькония,Эстус,9.28,2218-12-24
1348,Фалькония,Эстус,9.28,2218-12-25


In [27]:
baseline_model_df = pd.merge(baseline_date_df,
                             df_modelling,
                             how='left',
                             on=['place', 'product', 'date'])

In [28]:
baseline_model_df['price_comparison'] = np.where(
    baseline_model_df['own_price'] >
    baseline_model_df['competitors_mean_price'], 'Higher', 'Lower')

In [29]:
# установим порядок для категориальных признаков
baseline_model_df = baseline_model_df[
    [col for col in baseline_model_df if col != 'own_price'] + ['own_price']]

In [30]:
baseline_model_df = baseline_model_df[
    [col for col in baseline_model_df if col != 'price_comparison'] +
    ['price_comparison']]

In [31]:
baseline_model_df

,place,product,date,cost,competitors_mean_price,hot,rain,snow,month_name,season,own_price,price_comparison
0,Анор Лондо,Целебные травы,2218-09-28,6.950551,9.764069,0,0,0,September,Fall,13.63,Higher
1,Анор Лондо,Целебные травы,2218-09-29,6.945553,9.796939,1,0,0,September,Fall,13.63,Higher
2,Анор Лондо,Целебные травы,2218-09-30,6.926053,9.648104,0,0,0,September,Fall,13.63,Higher
3,Анор Лондо,Целебные травы,2218-10-01,6.902003,9.573142,0,1,0,October,Fall,13.63,Higher
4,Анор Лондо,Целебные травы,2218-10-02,6.955854,9.638593,0,0,0,October,Fall,13.63,Higher
...,...,...,...,...,...,...,...,...,...,...,...,...
1345,Фалькония,Эстус,2218-12-22,5.309055,15.655683,0,0,0,December,Winter,9.28,Lower
1346,Фалькония,Эстус,2218-12-23,5.264601,15.582679,0,0,0,December,Winter,9.28,Lower
1347,Фалькония,Эстус,2218-12-24,5.285202,15.597812,0,0,0,December,Winter,9.28,Lower
1348,Фалькония,Эстус,2218-12-25,5.280859,15.655546,0,0,1,December,Winter,9.28,Lower


- *Видно, что бейзлайн не выполняет условия по цене в 1.2 выше, чем у конкурентов, исправим это*

In [32]:
# Условие для- *Видно, что бейзлайн не выполняет условия по цене в 1.2 выше, чем у конкурентов, исправим это* изменения значений
condition = baseline_model_df['price_comparison'] == 'Higher'

# Изменение значений в столбце 'own_price'
baseline_model_df.loc[condition, 'own_price'] = baseline_model_df.loc[
    condition, 'competitors_mean_price'] * 1.2

In [33]:
baseline_model_df['amount'] = catb_model.predict(
    baseline_model_df.drop(['date'], axis=1))

In [34]:
# проверка
baseline_model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   place                   1350 non-null   object        
 1   product                 1350 non-null   object        
 2   date                    1350 non-null   datetime64[ns]
 3   cost                    1350 non-null   float64       
 4   competitors_mean_price  1350 non-null   float64       
 5   hot                     1350 non-null   int64         
 6   rain                    1350 non-null   int64         
 7   snow                    1350 non-null   int64         
 8   month_name              1350 non-null   category      
 9   season                  1350 non-null   category      
 10  own_price               1350 non-null   float64       
 11  price_comparison        1350 non-null   object        
 12  amount                  1350 non-null   float64 

In [35]:
baseline_model_df

,place,product,date,cost,competitors_mean_price,hot,rain,snow,month_name,season,own_price,price_comparison,amount
0,Анор Лондо,Целебные травы,2218-09-28,6.950551,9.764069,0,0,0,September,Fall,11.716883,Higher,36.516783
1,Анор Лондо,Целебные травы,2218-09-29,6.945553,9.796939,1,0,0,September,Fall,11.756327,Higher,48.989987
2,Анор Лондо,Целебные травы,2218-09-30,6.926053,9.648104,0,0,0,September,Fall,11.577725,Higher,35.885735
3,Анор Лондо,Целебные травы,2218-10-01,6.902003,9.573142,0,1,0,October,Fall,11.487771,Higher,54.527127
4,Анор Лондо,Целебные травы,2218-10-02,6.955854,9.638593,0,0,0,October,Fall,11.566311,Higher,40.311582
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,Фалькония,Эстус,2218-12-22,5.309055,15.655683,0,0,0,December,Winter,9.280000,Lower,57.843820
1346,Фалькония,Эстус,2218-12-23,5.264601,15.582679,0,0,0,December,Winter,9.280000,Lower,58.291761
1347,Фалькония,Эстус,2218-12-24,5.285202,15.597812,0,0,0,December,Winter,9.280000,Lower,58.305068
1348,Фалькония,Эстус,2218-12-25,5.280859,15.655546,0,0,1,December,Winter,9.280000,Lower,68.669885


## Eval baseline

In [36]:
baseline_model_df[
    'profit'] = baseline_model_df['own_price'] * baseline_model_df[
        'amount'] - baseline_model_df['cost'] * baseline_model_df['amount']

In [37]:
evaluation_df['profit_baseline'] = baseline_model_df['profit'].sum()

In [38]:
baseline_model_df['profit'].sum()

541522.1231377954

# Final result

In [47]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)
evaluation_df.style.highlight_max(axis=1, color='lightgreen')

,profit_own_algo,profit_baseline
0,789793.714942,541522.123138


In [40]:
evaluation_df['profit_own_algo'] - evaluation_df['profit_baseline']

0   248271.591804
dtype: float64

In [41]:
evaluation_df['profit_baseline'] / evaluation_df['profit_own_algo']

0   0.685650
dtype: float64

# Вывод

- *Что и требовалось доказать, наш алгоритм получает прибыли на 31% больше, чем бейзлайн. А именно **248271.59 золотых** или **24827159 серебряных** монет.*